In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

from skimage import filters 
from skimage.morphology import disk, remove_small_holes, remove_small_objects

from PIL import Image

In [ ]:
def thr_iba1_apotome (img, saveto , filename):
    enhanced = filters.unsharp_mask (img , radius = 20.0 , amount = 2 ) #reduces background
    otsu_thr = filters.threshold_otsu (enhanced) #set threshold with otsu
    otsu = enhanced > otsu_thr 
    mask = filters.median (otsu , disk (2)) 
    
    im = Image.fromarray (mask)
    im.save (saveto + "/" + filename.replace ("tif" ,"png" ))
    print ("Titel:\t" , saveto + "/" + filename.replace ("tif" ,"png" ))

In [ ]:
def thr_iba1_confocal (img, saveto, filename):
    sharpened = filters.gaussian (img)
    thr = filters.threshold_otsu (sharpened)
    otsu = (sharpened > thr)*1
    mask = (filters.median (otsu , disk (3)))*255
    mask = np.uint8 (mask)
    
    im = Image.fromarray (mask)
    im.save (saveto + "/" + filename.replace ("tif" ,"png" ))

In [ ]:
def get_tissue (nf_img, saveto, filename):
    nf_blur = filters.gaussian(nf_img, sigma=10)
    thr = filters.threshold_li(nf_blur)
    tissue = (nf_blur>thr)*1
    tissue = remove_small_holes(tissue, area_threshold=700)
    tissue = remove_small_objects (tissue, min_size=700)
    
    im = Image.fromarray (tissue)
    im.save (saveto + "/" + filename.replace ("tif" ,"png" ))
    print ("Tissue:\t" , saveto + "/" + filename.replace ("tif" ,"png" ))

In [ ]:
def walkthrough (rootdir, fct):
    for dirpath , dirnames , filenames in os.walk (rootdir) : 
        #thresholding for only iba1
        if dirpath.endswith ("iba1"):
            subdir = os.path.join (dirpath, dirname)
            saveto = subdir.replace ("iba1" , "iba1_pred/masks") 
            print (saveto)
            #make folders to save masks into
            os.makedirs(saveto) if not os.path.exists(saveto) else None
            
            #apply function to all the images in each folder
            for dirpath , dirnames , filenames in os.walk (subdir): 
                for filename in filenames: 
                    img_f = os.path.join (dirpath, filename)
                    img = plt.imread (img_f) 
                    
                    fct (img, saveto , filename) #creates and saves mask
                    ##or thr_iba1_confocal
                    
                    print ("\n")

In [ ]:
rootdir_A = r"D:\IF\CCI - Apotom\Females\Tifs\F1_NF-GFAP-Iba1"
rootdir_C = r"E:\IF\CCI - Confocal\F5_males\Tifs_max"

walkthrough (rootdir_A, thr_iba1_apotome)
walkthrough (rootdir_C, thr_iba1_confocal)